In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.metrics import classification_report, confusion_matrix
import shutil

In [8]:
# Defining constants
DATASET_DIR = "./data/Garbage_classification"
OUTPUT_YOLO_DIR = "./data/yolo"  # Path for YOLO-compatible data
YOLO_IMG_SIZE = 416  # Image size for YOLO


In [3]:
# Class names and indices
class_names = sorted(os.listdir(DATASET_DIR))  # Ensure proper alphabetical order
class_indices = {class_name: idx for idx, class_name in enumerate(class_names)}
print("Class Names and Indices:", class_indices)

Class Names and Indices: {'biological': 0, 'cardboard': 1, 'glass': 2, 'metal': 3, 'paper': 4, 'plastic': 5, 'trash': 6}


In [4]:
# YOLO Data Preparation
def prepare_yolo_data(dataset_dir, output_dir, class_indices):
    os.makedirs(output_dir, exist_ok=True)
    for class_name, class_id in class_indices.items():
        class_path = os.path.join(dataset_dir, class_name)
        for image_name in os.listdir(class_path):
            if image_name.endswith(('.jpg', '.png', '.jpeg')):
                # Copy image to output directory
                src_image = os.path.join(class_path, image_name)
                dest_image = os.path.join(output_dir, image_name)
                shutil.copy(src_image, dest_image)
                
                # Create YOLO annotation file
                label_file = os.path.splitext(image_name)[0] + ".txt"
                label_path = os.path.join(output_dir, label_file)
                with open(label_path, "w") as f:
                    # YOLO annotation format: class_id x_center y_center width height (normalized)
                    f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")  # Example: whole image bounding box

prepare_yolo_data(DATASET_DIR, OUTPUT_YOLO_DIR, class_indices)
print(f"YOLO-compatible dataset prepared in {OUTPUT_YOLO_DIR}")

YOLO-compatible dataset prepared in ./data/yolo


In [5]:
import os
import random
import shutil

# Define paths
YOLO_IMAGES_DIR = "./data/yolo/images"
YOLO_LABELS_DIR = "./data/yolo/labels"  # Path for YOLO labels directory
TRAIN_SPLIT = 0.8  # Percentage of data for training

class_names = sorted(os.listdir(DATASET_DIR))  # Assumes folders are class names
class_indices = {class_name: idx for idx, class_name in enumerate(class_names)}
print("Class Indices:", class_indices)

# Ensure output directories exist
os.makedirs(os.path.join(YOLO_IMAGES_DIR, "train"), exist_ok=True)
os.makedirs(os.path.join(YOLO_IMAGES_DIR, "val"), exist_ok=True)
os.makedirs(os.path.join(YOLO_LABELS_DIR, "train"), exist_ok=True)
os.makedirs(os.path.join(YOLO_LABELS_DIR, "val"), exist_ok=True)

# Iterate through class directories
for class_name, class_id in class_indices.items():
    class_path = os.path.join(DATASET_DIR, class_name)
    if not os.path.isdir(class_path):
        continue  # Skip non-directory files

    # Collect all images
    images = [f for f in os.listdir(class_path) if f.endswith(('.jpg', '.png'))]

    # Shuffle and split the data
    random.shuffle(images)
    split_idx = int(len(images) * TRAIN_SPLIT)
    train_images = images[:split_idx]
    val_images = images[split_idx:]

    # Process images for training and validation
    for subset, subset_name in [(train_images, "train"), (val_images, "val")]:
        for img_file in subset:
            # Source image path
            img_src = os.path.join(class_path, img_file)

            # Destination paths
            img_dest = os.path.join(YOLO_IMAGES_DIR, subset_name, img_file)
            label_dest = os.path.join(YOLO_LABELS_DIR, subset_name, os.path.splitext(img_file)[0] + ".txt")

            # Copy image
            shutil.copy(img_src, img_dest)

            # Write YOLO annotation file
            with open(label_dest, "w") as f:
                f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")  # Full image bounding box for class
                # `class_id` corresponds to the folder name

print("Dataset split and labeled for YOLO.")


Class Indices: {'biological': 0, 'cardboard': 1, 'glass': 2, 'metal': 3, 'paper': 4, 'plastic': 5, 'trash': 6}
Dataset split and labeled for YOLO.


In [6]:
from ultralytics import YOLO





In [ ]:
# retraining with new class
# Load the pre-trained YOLOv8 model
model = YOLO('./runs/detect/train13/weights/best.pt')  # Use the nano model or replace with your desired pre-trained model


# Train on the new dataset
model.train(
    data='configuration.yaml',  # Updated YAML file
    epochs=35,                    # Number of epochs
    imgsz=416,                    # Image size
    batch=25,                     # Batch size
    device=0,                     # Use GPU (set to 'cpu' if GPU is unavailable)
    resume=True                    # Retain pre-trained weights for existing classes
)

In [7]:
model = YOLO('yolov8n.pt')  # Pre-trained weights

# Train the model
model.train(
    data="configuration.yaml",  # Path to YAML file defining your dataset
    epochs=50,                   # Number of epochs
    imgsz=416,                   # Image size
    batch=20,                    # Batch size
    device=0                     # Use GPU (set to 'cpu' if GPU not available)
)

New https://pypi.org/project/ultralytics/8.3.47 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.37  Python-3.11.9 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=configuration.yaml, epochs=50, time=None, patience=100, batch=20, imgsz=416, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train20, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_fr

train: Scanning E:\MyProjects\Garbage management system\ML\data\yolo\labels\train... 5791 images, 0 backgrounds, 0 corr


train: New cache created: E:\MyProjects\Garbage management system\ML\data\yolo\labels\train.cache


val: Scanning E:\MyProjects\Garbage management system\ML\data\yolo\labels\val... 4105 images, 0 backgrounds, 0 corrupt:


val: New cache created: E:\MyProjects\Garbage management system\ML\data\yolo\labels\val.cache
Plotting labels to runs\detect\train20\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000909, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.00046875), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 416 train, 416 val
Using 8 dataloader workers
Logging results to runs\detect\train20
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       4105       4105      0.816      0.828      0.752      0.752



50 epochs completed in 1.911 hours.
Optimizer stripped from runs\detect\train20\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train20\weights\best.pt, 6.2MB

Validating runs\detect\train20\weights\best.pt...
Ultralytics 8.3.37  Python-3.11.9 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)
Model summary (fused): 168 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 103/103 [00:


                   all       4105       4105      0.817      0.827      0.756      0.756
             cardboard        820        820      0.999      0.889      0.938      0.938
                 glass        598        598      0.836      0.788       0.74       0.74
                 metal        645        645      0.794      0.803      0.738      0.738
                 paper        525        525      0.731      0.663      0.587      0.587
               plastic        604        604      0.718      0.758      0.636      0.636
                 trash        445        445      0.729      0.888      0.699      0.699
            biological        468        468      0.913          1      0.958      0.958
Speed: 0.3ms preprocess, 2.8ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs\detect\train20


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000029041AE3890>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047

In [7]:
import torch

print(torch.cuda.is_available())

True


In [9]:
metrics = model.val()
print(metrics)

Ultralytics 8.3.37  Python-3.11.9 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)
Model summary (fused): 168 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning E:\MyProjects\Garbage management system\ML\data\yolo\labels\val.cache... 3351 images, 0 backgrounds, 0 co
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 168/168 [01:


                   all       3351       3351      0.695      0.705      0.621       0.62
             cardboard        738        738      0.991      0.795      0.852      0.851
                 glass        429        429      0.612      0.564      0.536      0.536
                 metal        448        448      0.608      0.621      0.542      0.542
                 paper        461        461      0.594      0.516      0.386      0.386
               plastic        537        537      0.598      0.665      0.528      0.527
                 trash        332        332       0.61      0.783      0.585      0.585
            biological        406        406      0.852      0.993      0.915      0.915
Speed: 0.6ms preprocess, 7.2ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs\detect\train192
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6])
box: ultralytics.utils.metrics.Metric object
confusion_matrix


image 1/6 E:\MyProjects\Garbage management system\ML\single_pred\cardboard.jpg: 352x416 1 cardboard, 109.8ms
image 2/6 E:\MyProjects\Garbage management system\ML\single_pred\glass.jpg: 416x384 1 paper, 136.4ms
image 3/6 E:\MyProjects\Garbage management system\ML\single_pred\metal.jpg: 416x416 1 metal, 25.6ms
image 4/6 E:\MyProjects\Garbage management system\ML\single_pred\paper.jpg: 256x416 1 cardboard, 77.6ms
image 5/6 E:\MyProjects\Garbage management system\ML\single_pred\plastic.jpg: 288x416 1 paper, 138.8ms
image 6/6 E:\MyProjects\Garbage management system\ML\single_pred\trash.jpg: 416x416 1 paper, 25.0ms
Speed: 20.2ms preprocess, 85.5ms inference, 30.9ms postprocess per image at shape (1, 3, 416, 416)
Results saved to runs\detect\train133
6 labels saved to runs\detect\train133\labels


In [11]:
# Directory containing images
images_dir = "./single_pred/new"

# Predict on all images in the directory
results = model.predict(source=images_dir, imgsz=416, save=True, save_txt=True)


image 1/5 E:\MyProjects\Garbage management system\ML\single_pred\new\glass.jpg: 416x288 1 glass, 32.1ms
image 2/5 E:\MyProjects\Garbage management system\ML\single_pred\new\images.jpg: 320x416 1 trash, 201.3ms
image 3/5 E:\MyProjects\Garbage management system\ML\single_pred\new\paper.png: 416x416 1 cardboard, 35.2ms
image 4/5 E:\MyProjects\Garbage management system\ML\single_pred\new\plastic.jpg: 416x256 1 cardboard, 1 plastic, 38.9ms
image 5/5 E:\MyProjects\Garbage management system\ML\single_pred\new\trash.jpg: 416x416 1 cardboard, 23.4ms
Speed: 71.2ms preprocess, 66.2ms inference, 6.0ms postprocess per image at shape (1, 3, 416, 416)
Results saved to runs\detect\train135
5 labels saved to runs\detect\train135\labels


In [6]:
model = YOLO('./runs/detect/train13/weights/best.pt')